In [13]:
from dataset import FurnitureDataset
from models import *
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import argparse
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Define arguments
encoding_dim = 32
img_size = 64
mode = 'encode'
model_name = 'chairs_lamps'

# Device configuration
device = torch.device('cuda')

# Prepare image
furnitures = model_name.split('_')
furniture_dataloader = []
for furniture_type in furnitures:
    dataset = FurnitureDataset(furniture_type, img_size)
    dataloader = DataLoader(dataset)
    furniture_dataloader.append(dataloader)

# Load encoder model
encoder = ConvAE_encoder(encoding_dim).to(device)
encoder_path = os.path.join('./weights', f'{model_name}_encoder.pth')
encoder.load_state_dict(torch.load(encoder_path))

# Create latent space
# Create dictionary for each furniture type so that the latent vector distribution can be analyzed separately
latent_vector = [{'vector': [], 'label': []} for _ in range(len(furnitures))]

for furniture_idx, dataloader in enumerate(furniture_dataloader):
    progress = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, data in progress:
        # Encode
        furniture_item = data['item'].to(device)
        #furniture_item = furniture_item.view(furniture_item.shape[0], -1)
        furniture_item_encoded = encoder(furniture_item)

        # Populate latent space
        furniture_type = data['type'][0]
        latent_vector[furniture_idx]['vector'].append(furniture_item_encoded.cpu().detach().numpy())
        latent_vector[furniture_idx]['label'].append(furniture_type)

        # Verbose
        progress.set_description(
        f'[{i}/{len(dataloader) - 1}]')

[6/22052]:   0%|                                                                             | 0/22053 [00:00<?, ?it/s]

Loading chairs dataset...
chairs dataset loaded successfully.
Loading lamps dataset...
lamps dataset loaded successfully.


[32401/32401]: 100%|████████████████████████████████████████████████████████████| 32402/32402 [02:41<00:00, 200.39it/s]


In [15]:
img_shape = (latent_vector[0]['vector'][0].shape)
flatten_shape = np.prod(img_shape)

# latent space
latent_vector_np = [np.vstack( latent_vector[i]['vector'] ) for i in range(len(furnitures))]
latent_vector_pd = [pd.DataFrame(data=latent_vector_np[i]) for i in range(len(furnitures))]

# Load decoder model
decoder = ConvAE_decoder(encoding_dim).to(device)
decoder_path = os.path.join('./weights', f'{model_name}_decoder.pth')
decoder.load_state_dict(torch.load(decoder_path))

print(encoder)
print(decoder)

# Input latent vector
vector_i = latent_vector_pd[0].mean()
vector_f = latent_vector_pd[1].mean()
num_vectors = 10
unit_vector = (vector_f-vector_i)/num_vectors

latent_vector_inputs = [torch.from_numpy((vector_i+i*unit_vector).to_numpy()).to(device) for i in range(num_vectors)]

print(latent_vector_inputs[0])

# latent_vector_pd[0].mean()
# latent_vector_input_np = ((latent_vector_pd[0].mean() + latent_vector_pd[1].mean())/2).to_numpy()
# latent_vector_input = torch.from_numpy(latent_vector_input_np).to(device)

imgs = []
for idx, latent_vector_input in enumerate(latent_vector_inputs):
    output = decoder(latent_vector_input)
    output_reshape = torch.reshape(output, (3, img_size, img_size)).cpu().detach()
    im = transforms.ToPILImage()(output_reshape).convert("RGB")
    imgs.append(im)

imgs[0].save('out.gif', save_all=True, append_images=imgs[1:], loop=0)

ConvAE_encoder(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): ReLU()
    (7): Linear(in_features=16384, out_features=32, bias=True)
    (8): ReLU()
  )
)
ConvAE_decoder(
  (model): Sequential(
    (0): Linear(in_features=32, out_features=16384, bias=True)
    (1): ReLU()
    (2): Unflatten(dim=1, unflattened_size=(16, 32, 32))
    (3): ReLU()
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Sigmoid()
  )
)
tensor([2.6594, 3.7237, 2.7260, 2.2749, 2.8533, 0.0000, 3.4094, 2.8406, 3.3585,
        0.0000, 0.0000, 3.5891

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [47]:
latent_vector_pd = pd.DataFrame(data=latent_vector_np[0])

ValueError: Must pass 2-d input. shape=(22053, 16, 32, 32)

In [20]:
print(len(latent_vector_np))
print(latent_vector_np[0].shape)

2
(17100, 32)
